Using the elliptic curve notebook + binary hashing notebook


page 19 in the monero doc

each txn is a signed (by private key, of the hash?) of:

  * public key of signer (e.g. spender/sender)
  * public key of receiver
  * the amount in question
  * previous hash
  * => all four will be bitwise concatenated, hashed, and signed (via private key)
  
  
This will get represented by a dataclass

Write queries by looking at the total amount for a given Key, returning 0 if the key has never been used before


=> Monero Post will build on this by introducing the OTA, use of two-key pairs, use of Pederson Commitments to hide the amounts (section 5.3), and ring signatures (close to how I thought the two-key pairs worked + Pederson Committments, section 5.4)

# Implementing Bitcoin From Scratch


## What is Bitcoin? The ten-feet-high version 

Bitcoin is a permission-less, trust-less way to transfer value. It does so by having a group of people who do not necesarily know or trust each other nonetheless agree to a standard ledger called a blockchain. Transactions are stamped to this blockchain and everybody agrees that this blockchain ledger has value. 


## What Problems do you need to Solve to Make this Work?

Two principal ones: Consensus & Open Verification. 

Consensus is the process by which the tens of thousands of servers on the Bitcoin network agree on which transactions should go into the blockchain. This will not be discussed here. 

What will be discussed is the problem of open verification: How do I know that person A agreed to send bitcoin to person B, openly where everybody can see, without me needing to trust that person A is telling the truth and without other people forging transactions?

That was a lot to say so let's describe it with a scenario. 

Alice wants to send Bob bitcoin. She does so by going to the blockchain ledger and _signs_ a transaction. This signature is open, anybody can read it, but only Alice can write that particular signature. Nobody else is able to write Alice's signature. Bob can then verify that she actually sent him money -- e.g. his bitcoin balance -- openly without Alice giving up any information. 

In the above scenario, the private key becomes the way Alice can sign this transaction (_uniquely!_) and the public key becomes the way we can verify it. 

But how does Bitcoin do this? How do we sign these signatures and generate these keys?


## Cryptography to the Rescue

To solve this problem, we need to solve a related one: how can Bob and Alice exchange messages such that only they can read it even if people inbetween can read the encrypted message?




## Enter: Ellitpic Cryptography

Elliptic Cryptography solves all these problems for us. We will start with how this will work and then implement each step at the end. 

Elliptic cryptography uses an elliptic function to generate a unique set of points. The function is parameterized with _a_ and _b_ and looks like:

$$
E\ =\ \{(x,y) | y^2 = x^3 + ax + b \} \cup \{ \mathcal{O} \}
$$

Where $ \mathcal{O} $ is the point at infinity -- more on this later. 

This function 

In [44]:
import random
from typing import Union, Callable, Tuple

class EllipticPoint(object):
    
    def __init__(self, 
        x: int, y: int = None, 
        a: int = 486662, b: int = 1, c: int = 0,
        prime: int = ((2**255) - 19)):
        '''
        For storing Elliptic Points corresponding to elements:
        
        y^2 = x^3 + ax^2 + bx + c
        
        Parameters a through d default to the ed25519 curve:
        
        y^2 = x^3 + 486662x^2 + x 
        
        where a=486662, b = 1, & c = 0
        
        :param x: the integer for the x coordinate of the point
        :param y: the integer for the y cooirdinate. If none, it will calculate 
            this based on the curve parameters provided (a - c)
        :param a: corresponds to the elliptic function (see above)
        :param b: corresponds to the elliptic function (see above)
        :param c: corresponds to the elliptic function (see above)
        :param prime: prime to use for modulo operations, defaults to ((2**255) - 19) 
            from the ed25519 curve
        '''
        self.x = x;
        self._curve = EllipticPoint._curve_factory(a, b, c, prime)
        if y:  self.y = y;
        else:  self.y = self._curve(x);
        self.a, self.b, self.c = a,b,c;
        self.prime = prime;
        
    def to_tuple(self):
        return (self.x, self.y)
        
    def __repr__(self):
        return "<EllipticPoint: %r,%r>" % (self.to_tuple())
    
    def __rmul__(self, scalar):
        return self.__mul__(scalar);
    
    def __lmul__(self, scalar):
        return self.__mul__(scalar);
    
    def __mul__(self, scalar):
        '''
        Elliptic Multiply two points together. Both must have the same parameters. 
        
        >>> G = EllipticPoint(x=5, a=2, prime=17)
        >>> G * 10
        (7, 11)
        >>> G * 19
        (inf, inf)
        >>> 21 * G
        (6, 3)
        '''
        if not type(scalar) in [int]:
            try:
                scalar = int(scalar)
                if scalar == 0:  raise
            except:
                raise ValueError("Can only multiply against scalar values -- value of type %r passed in: %r" %(
                    type(scalar), scalar))
            
        _running = self.copy()
        i = 1
        doubling = True
        while i != scalar:
            print("Multiplying--", i, scalar)
            if _running.x == float('inf'):
                i += 1
                _running = self.copy()
                continue 
            
            if doubling:
                if i * 2 > scalar:  doubling = False
            
            if doubling:
                i *= 2
                _running += _running
            else:
                i += 1
                _running += self
        return _running
        
        
    def __add__(self, anotherObj):
        '''
        Given another point, it will add the two together and return the 
        corresponding point in an EllipticPoint object. Both must have the 
        same parameters
        
        >>> P = EllipticPoint(x=5, y=1, a=0, b=2, c=2, prime=17)
        >>> Q = EllipticPoint(x=5, y=1, a=0, b=2, c=2, prime=17)
        >>> P + Q
        <EllipticPoint: 6,3>

        >>> P = EllipticPoint(x=7, y=6, a=0, b=2, c=2, prime=17)
        >>> Q = EllipticPoint(x=5, y=1, a=0, b=2, c=2, prime=17)
        >>> P + Q
        <EllipticPoint: 7,11>
        ''' 
        x_p, y_p = self.x, self.y;
        x_q, y_q = anotherObj.x, anotherObj.y
        
        if not self._same_curve(anotherObj):
            raise ValueError("Both points do not correspond to the same curve")

        # point doubling
        if x_p == x_q and y_p == y_q:
            s_num = 3 * (x_p**2) + self.b
            s_denom = 2 * y_p
            s = (s_num % self.prime) * EllipticPoint._multiplicative_inverse(s_denom, self.prime) 
            x_r = (s**2) - (2 * x_p)
            y_r = (s*(x_p - x_r)) - y_p

        # adding vertical points -- infinite number allowed(?)
        elif x_p == x_q:
            return EllipticPoint(float('inf'), float('inf'))

        # standard addition
        else:
            s_num = (y_p - y_q)
            s_denom = (x_p - x_q)
            s = (s_num % self.prime) * EllipticPoint._multiplicative_inverse(s_denom, self.prime)
            x_r = (s**2) - (x_p + x_q)
            y_r = ( s * (x_p - x_r) ) - y_p

        to_ret = self.copy()
        to_ret.x = int(x_r % self.prime)
        to_ret.y = int(y_r % self.prime)
        return to_ret
    
    def copy(self):
        '''
        Return a perfect copy of this elliptic point.
        '''
        return EllipticPoint(
            x = self.x, y = self.y,
            a = self.a, b = self.b, c = self.c, prime = self.prime)
        
    def _same_curve(self, anotherObj) -> bool:
        '''
        Given a passed function _curve, this will return true if this curve 
        has the same parameters as the curve represented within the object. Not
        meant for use outside of internal functions.
        '''
        return (
            self.a == anotherObj.a 
            and self.b == anotherObj.b
            and self.c == anotherObj.c
            and self.prime == anotherObj.prime
        )
        
    @staticmethod
    def _curve_factory(a: int, b: int, c: int, prime: int) -> Callable[[int], int]:
        '''
        Returns a lambda function corresponding to the curve:
        
        y^2 = ( x^3 + ax^2 + bx + c ) % prime
        
        All parameters passed in correspond
        
        :returns: function corresponding to the curve
        '''
        return lambda x: ((x**3) + (a*(x**2)) + (b*x) + c) % prime;
     
    @staticmethod
    def _multiplicative_inverse(a: int, m: int) -> int:
        '''
        Taken with much love from https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/
        
        Given the inverse of (a % m)^-1, we try to find the gcd
        for a & m starting at 1 -- ax + my = 1.
        
        We take the modulo of both sides, knowing that my % m will always
        be zero.
        
        :param x: number
        :param m: the modulo for the inverse
        
        :returns: the multiplicative inverse
        
        >>> EllipticPoint._multiplicative_inverse(3, 11)
        4
        
        >>> EllipticPoint._multiplicative_inverse(3, 17)
        6
        
        >>> EllipticPoint._multiplicative_inverse(2, 17)
        9
        '''
        m0 = m
        y = 0
        x = 1

        if (m == 1):
            return 0

        while (a > 1):
            q = a // m

            t = m

            m = a % m
            a = t
            t = y

            y = x - q * y
            x = t

        # Make x positive
        if (x < 0):
            x = x + m0

        return x

In [32]:
EllipticPoint._multiplicative_inverse(3, 11)

4

In [2]:
G = EllipticPoint(x=5, y=1, a=2, b=2, c=2, prime=17)
print(G.y)
#print(10 * G)
#print(G * 19)
print(21 * G)  # should be 6,3

1
1 21 <EllipticPoint: 5,1>
2 21 <EllipticPoint: 6,3>
4 21 <EllipticPoint: 3,1>
8 21 <EllipticPoint: 13,7>
16 21 <EllipticPoint: 10,11>
17 21 <EllipticPoint: 6,14>
18 21 <EllipticPoint: 5,16>
19 21 <EllipticPoint: inf,inf>
20 21 <EllipticPoint: 5,1>
<EllipticPoint: 6,3>


In [3]:
P = EllipticPoint(x=5, y=1, a=2, prime=17)
Q = EllipticPoint(x=5, y=1, a=2, prime=17)
print(P)
P += Q
print(P)

<EllipticPoint: 5,1>
<EllipticPoint: 6,12>


In [ ]:
# secp256k1: y^2 = x^3 + 7
# ed25519: y^2 = x^3 + 486662x^2 + x

https://en.bitcoin.it/wiki/Elliptic_Curve_Digital_Signature_Algorithm

* `r` & `s` =>  represents the signature
* `k` (nonce) =>  random value that must be unique to this transaction, between 1 and n-1 (group order)
* `z`: hash of our message
* `d`: private key of sender
* `D`: public key of sender

Signing:

1. Generate point C: `(x,y) = k * G`  w/curve
2. Compute `r = x_c % n` 
3. Compute `s = k^-1 * (z + r * d)`, if s=0, generate another `k` and start over


Verification

1. `r` & `s` are between 1 and n-1
2. Compute `u1 = z * s^-1 % n` and `u2 = r * s^-1`
3. Compute `(x, y) = u1 * G + u2 * D`
4. if `r = x mod n`, then the signatures are valid, otherwise the check fails

[another link](https://andrea.corbellini.name/2015/05/30/elliptic-curve-cryptography-ecdh-and-ecdsa/)

In [46]:
from dataclasses import dataclass

prime = ((2**255) - 19)
a, b, c = 0, 2, 2
norder = ((2**255) - 19)  # could be computed by trying to multiply until we get infinity
# y^2 = x^3 + 2x + 2
generatorPoint = EllipticPoint(x=5, y=1, a=0, b=2, c=2, prime=prime)
nonce_generator = 11


@dataclass
class transaction:

    public_key_sender: int
    public_key_receiver: int
    amount: int
    message: str = ""  # hash of this is z

    nonce: int = None  # k in the above, between 1 and 18
    txn_id: int = None  # r in the above
    sf: int = 0  # s in the above, signature factor
    order: int = ((2**255) - 19)

    def generate(self) -> None:
        self.nonce = random.randint(1, self.order)
        x_public, _ = (self.nonce * generatorPoint).to_tuple()
        self.txn_id = 0
        while self.txn_id == 0:
            self.txn_id = x_public % self.order
        
        
    def sign(self, private_key_sender) -> None:
        self.sf = 0
        while self.sf == 0:
            self.sf = ( 
                (hash(self.message) + (self.txn_id * private_key_sender)) 
                / self.nonce 
            ) % self.order
        
    def verify(self) -> bool:
        # should be checking validity of txn_id and sf 
        # but skipping for now
        u1 = (self.nonce % self.order) * EllipticPoint._multiplicative_inverse(self.sf, self.order)
        u2 = (self.txn_id % self.order) * EllipticPoint._multiplicative_inverse(self.sf, self.order)
        
        #u1 = (hash(self.message) / self.sf) % self.order
        #u2 = (self.txn_id / self.sf) % self.order
        
        print("u1,u2", u1, u2);
        
        new_point = (generatorPoint * u1) + (
            u2
            * EllipticPoint(
                x=self.public_key_sender,
                y=self.public_key_receiver,
                a=0,
                b=2,
                c=2,
                prime=17,
            )
        )

        verify_id = new_point.x % norder

        print("verify id and txn_id", verify_id, self.txn_id)

        return self.txn_id == verify_id


In [34]:
sender = EllipticPoint(x=9, a=0, b=2, c=2, prime=17)
receiver = EllipticPoint(x=3, a=0, b=2, c=2, prime=17)

print(sender, receiver)  # their public values need to be calculated as scalar times the generator points

sender_public_key = sender.x * generatorPoint
receiver_public_key = receiver.x * generatorPoint

<EllipticPoint: 9,1> <EllipticPoint: 3,1>
1 9 <EllipticPoint: 5,1>
2 9 <EllipticPoint: 6,3>
4 9 <EllipticPoint: 3,1>
8 9 <EllipticPoint: 13,7>
1 3 <EllipticPoint: 5,1>
2 3 <EllipticPoint: 6,3>


txn => alice
sender => bob

```
Mp (Xm,Ym) = (Message Hash=Mh)* G(Xg,Yg)
Rp (Xr,Yr) = (Random Number=Rn)* G(Xg,Yg)
Pu (Xk,Yk) = (Xm)* Public Key(Xpub,Ypub)= (Xm)*(Private Key=Pr)* G(Xg,Yg)


Signature Factor=Sf=(Mh + Xr * Pr) / Rn mod p
Public Key(Xpub, Ypub)= (Private Key=Pr)* G(Xg,Yg) 
U1 = ((Message Hash = Mh))/Sf 
U2 = ((Xr))/Sf


// following can be reconstructed from the other two
// Xr of the sender ≡ Xr of the receiver then the signature is valid 
Rp (Xr,Yr) = (U1)* G(Xg,Yg)+ (U2)* Public Key(Xpub,YPub)
```


_______________________

Bitcoin Curve:

$$ y^2 = x^3 + 0x + 7 mod p=1.158 * 10^77 $$

Generator point is agreed upon ahead of time -- I think?